In [1]:
# 示例代码运行环境
%load_ext watermark
%watermark -p tensorflow,numpy -v -m

CPython 3.5.2
IPython 5.3.0

tensorflow 1.0.1
numpy 1.12.1

compiler   : GCC 4.9.2
system     : Linux
release    : 3.16.0-4-amd64
machine    : x86_64
processor  : 
CPU cores  : 4
interpreter: 64bit


In [2]:
import tensorflow as tf
import numpy as np

## RNN Demo (Numpy)

In [3]:
# assume input and state are 128-length vector
class RNNCell:
    def __init__(self, input_size, state_size):
        self.W = np.random.rand(input_size, state_size)
        self.U = np.random.rand(state_size, state_size)
        self.b = np.zeros((1, state_size))
        
    def forward(self, input_, state):
        return input_.dot(self.W) + state.dot(self.U) + self.b


In [4]:
rnn = RNNCell(128, 128)

In [5]:
input0 = np.random.rand(1, 128)

In [6]:
state0 = np.zeros((1, 128))
state1 = rnn.forward(input0, state0)
print(state1.shape)

(1, 128)


In [7]:
input1 = np.random.rand(1, 128)
state2 = rnn.forward(input1, state1)
print(state2.shape)

(1, 128)


## RNN Demo (Tensorflow)

In [8]:
tf.reset_default_graph()

In [9]:
vocab_size = 100
word_embedding_dim = 128

In [10]:
cell = tf.contrib.rnn.BasicRNNCell(128)

In [11]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [12]:
data = np.asarray(list(chunks(range(30), 3)))

In [13]:
print(data)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]
 [15 16 17]
 [18 19 20]
 [21 22 23]
 [24 25 26]
 [27 28 29]]


In [14]:
word_embedding = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim]))

In [15]:
input_embeds = tf.nn.embedding_lookup(word_embedding, data[:, [0, 1]])

In [16]:
output, states = tf.nn.dynamic_rnn(cell, input_embeds, dtype=tf.float32)
print(output, states)

Tensor("rnn/transpose:0", shape=(10, 2, 128), dtype=float32) Tensor("rnn/while/Exit_2:0", shape=(?, 128), dtype=float32)


In [17]:
output_flat = tf.reshape(output, (-1, 128))

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_val = output.eval()
    states_val = states.eval()
    print(output_flat.eval().shape)
    print(output_val.shape)
    print(output_val.reshape((-1, 128)).shape)
    print(states_val.shape)
    print((output_val[:,1,:] == states_val).all())

(20, 128)
(10, 2, 128)
(20, 128)
(10, 128)
True


## RNN Demo（处理变长数据）

In [19]:
data = [[1, 2], [1, 2, 3], [1, 2, 3, 4, 5]]

In [20]:
data

[[1, 2], [1, 2, 3], [1, 2, 3, 4, 5]]

In [21]:
data_padding = np.asarray([[1, 2, 0, 0, 0], [1, 2, 3, 0, 0], [1, 2, 3, 4, 5]])

In [22]:
data_padding

array([[1, 2, 0, 0, 0],
       [1, 2, 3, 0, 0],
       [1, 2, 3, 4, 5]])

Padding 的问题:
1. RNN 进化到一个『不对』的状态
2. 最长的句子可能很长，影响性能
3. 损失函数没法安排，多了很多『假』Label

In [23]:
tf.reset_default_graph()
vocab_size = 100
word_embedding_dim = 128

In [24]:
cell = tf.contrib.rnn.BasicRNNCell(128)
word_embedding = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim]))

In [25]:
input_embeds = tf.nn.embedding_lookup(word_embedding, data_padding[:, 0:4])

In [26]:
data_padding[:, 0:4]

array([[1, 2, 0, 0],
       [1, 2, 3, 0],
       [1, 2, 3, 4]])

In [27]:
print(data_padding[:, 0:4])

[[1 2 0 0]
 [1 2 3 0]
 [1 2 3 4]]


In [29]:
output, states = tf.nn.dynamic_rnn(cell, input_embeds, dtype=tf.float32, sequence_length=[2, 3, 4])

In [30]:
output

<tf.Tensor 'rnn/transpose:0' shape=(3, 4, 128) dtype=float32>

In [31]:
output_flat = tf.reshape(output, [-1, 128])

In [32]:
output_flat

<tf.Tensor 'Reshape:0' shape=(12, 128) dtype=float32>

把原来的 cost 和 mask 函数相乘，作为新的 cost 给优化器进行优化，即可忽略这些无用的 labels

In [33]:
labels = tf.reshape(data_padding[:, 1:5], [-1])
mask = tf.sign(labels)
with tf.Session() as sess:
    print(mask.eval())

[1 0 0 0 1 1 0 0 1 1 1 1]


In [34]:
labels

<tf.Tensor 'Reshape_1:0' shape=(12,) dtype=int64>

In [ ]:
cost = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output_flat)

In [ ]:
tf.matmul(output_flat, 

## 用于生成时的一些提示

cell 的 zero_state 函数可以产生该 cell 的初始状态（对 BasicRNNCell 来说就是一个 0 矩阵）
cell 本身也可以调用 (Python 里的 __call__ 方法，用于产生单步的状态变化

tf.slice 可以用于取 tensor 的某一列等

In [ ]:
tf.reset_default_graph()
vocab_size = 100
word_embedding_dim = 128
word_embedding = tf.Variable(tf.random_uniform([vocab_size, word_embedding_dim]))
input_embeds = tf.nn.embedding_lookup(word_embedding, data_padding[:, 0:4])

In [ ]:
input_embeds.shape

In [ ]:
## 取第一个词
single_input_embeds = tf.slice(input_embeds, [0, 0, 0], [3, 1, 128])
print(single_input_embeds.shape)

In [ ]:
# Cell 走一步
output, state = cell(tf.reshape(single_input_embeds, (3, 128)), cell.zero_state(3, dtype=tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_val = output.eval()
    state_val = state.eval()
    print((output_val == state_val).all())